# General notebook preparation

In [2]:
# Standard library imports
import os
import sys
import yaml

In [3]:
# Set constants for environment
IN_COLAB = 'google.colab' in sys.modules
IN_LOCAL_CONTAINER = 'HOMELAB' in os.environ
env = {
    'PROJECT_NAME': 'LVGL UI Detector',
    'PROJECT_TAG': 'lvgl-ui-detector',
    'IN_COLAB': IN_COLAB,
    'IN_LOCAL_CONTAINER': IN_LOCAL_CONTAINER,
    'DIRS': {},
    'FILES': {},
    'ENV': os.environ,
}
required_modules = ['ultralytics', 'clearml']

# Environment setups

In [4]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive') # Mount Google Drive
    env['DIRS']['root'] = os.path.join("/content/drive/MyDrive/project")
elif IN_LOCAL_CONTAINER:
    # Local container already comes with the required modules pre-installed
    env['DIRS']['root'] = "/usr/src"
else:
    # Local development environment needs to have the required modules installed
    env['DIRS']['root'] = os.path.curdir

In [5]:
# Import all dependencies
missing_deps = False
try:
    from ultralytics import YOLO
    from clearml import Task, TaskTypes, Dataset, StorageManager, Logger
    import yaml
except ImportError as e:
    print(f"Failed to import required dependencies: {e}")
    missing_deps = True
    print("Run the next cell to install the required dependencies and then restart the runtime.")

In [6]:
if missing_deps:
    %pip install -q {*required_modules}

In [7]:
# Show the used versions of the required modules
for module in required_modules:
    %pip show {module}

Name: ultralytics
Version: 8.1.47
Summary: Ultralytics YOLOv8 for SOTA object detection, multi-object tracking, instance segmentation, pose estimation and image classification.
Home-page: 
Author: Glenn Jocher, Ayush Chaurasia, Jing Qiu
Author-email: 
License: AGPL-3.0
Location: /home/rini-debian/git-stash/lvgl-ui-detector/.venv/lib/python3.11/site-packages
Requires: matplotlib, opencv-python, pandas, pillow, psutil, py-cpuinfo, pyyaml, requests, scipy, seaborn, thop, torch, torchvision, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.
Name: clearml
Version: 1.15.1
Summary: ClearML - Auto-Magical Experiment Manager, Version Control, and MLOps for AI
Home-page: https://github.com/allegroai/clearml
Author: ClearML
Author-email: support@clear.ml
License: Apache License 2.0
Location: /home/rini-debian/git-stash/lvgl-ui-detector/.venv/lib/python3.11/site-packages
Requires: attrs, furl, jsonschema, numpy, pathlib2, Pillow, psutil, pyjwt, pyparsing, python-

In [8]:
# Configure folders
env['DIRS']['data'] = os.path.join(env['DIRS']['root'], "datasets")

In [9]:
# Query ClearML for available datasets
datasets = Dataset.list_datasets(env['PROJECT_NAME'], partial_name="UI Randomizer", tags=[env['PROJECT_TAG']], only_completed=True)
# Store dataset filenames per dataset
env['DATASETS'] = {}
for dataset in datasets:
    env['FILES'][dataset['id']] = Dataset.get(dataset['id']).list_files("*.yaml")
    env['DATASETS'][dataset['id']] = dataset


In [10]:
# Debugging output

print(env['FILES'])
print(env['DATASETS'])

{'d1c4ec5c8f7e4492ab7b2a8d0b584c88': ['ui_randoms.yaml'], 'e7f05a856ca34cd88940c4b8774f4c45': ['ui_randoms.yaml'], '50e10f640d7548458d9c38ab9aac571b': ['ui_randoms.yaml'], 'e113a14fec574025a15a5c9868746c4b': ['dataset_upload_test.yaml'], 'c520737c392c47608ffd52f33fc593f6': ['dataset_upload_test.yaml'], '12f2a7ee7a7f45b8866d2c5f75e92413': ['dataset_upload_test.yaml'], '640fc2e67b3b4b498fe6d9b1da764911': ['dataset_upload_test.yaml'], 'cef7f5e021484b089a5eccf0c014d150': ['dataset_upload_test.yaml'], '28b1c17f1fef47a793f9ac086b2d6b10': ['dataset_upload_test.yaml'], 'af33141c123843d4bf0c50db76635bba': ['dataset_upload_test.yaml'], '2b561087cf974658b576fbd17a87bc87': ['dataset_upload_test.yaml'], '366058db107e485ca273a11601e68e7a': ['dataset_upload_test.yaml'], 'f4d7bdcb58534f1ebbc5c5578b6801a5': ['dataset_upload_test.yaml'], '90648781cf4f4848a2a66ca572e1edf5': ['dataset_upload_test.yaml'], 'a4b6cc7161334df1ad94143a819842d6': ['dataset_upload_test.yaml'], '853a677148c5417e8cc003d6761236e3': 

# Helper functions

In [11]:
def download_dataset(id: str, overwrite: bool = True):
    dataset = Dataset.get(id)
    return dataset.get_mutable_local_copy(env['DIRS']['data'], overwrite=overwrite)

In [12]:
def fix_dataset_path(file: str, replacement_path: str):
    # Replace path in dataset file to match current environment
    with open(file, 'r+') as f:
        dataset_content = yaml.safe_load(f)
        dataset_content['path'] = replacement_path
        print(f"Original dataset:\n{dataset_content}")
        f.seek(0)
        yaml.dump(dataset_content, f)
        f.truncate()
        f.seek(0)
        print(f"Adjusted dataset:\n{f.read()}")
        return dataset_content

In [13]:
def training_task(model_variant: str, dataset_id: str, args: dict, project: str = "LVGL UI Detector"):
    # Download & modify dataset
    env['DIRS']['target'] = download_dataset(dataset_id)
    dataset_file = os.path.join(env['DIRS']['target'], env['FILES'][dataset_id][0])
    dataset_content = fix_dataset_path(dataset_file, env['DIRS']['target'])
    args['data'] = os.path.join(env['DIRS']['target'], env['FILES'][dataset_id][0])
    # Create a ClearML Task
    task = Task.init(
        project_name="LVGL UI Detector",
        task_name=f"Train {model_variant} ({env['DATASETS'][dataset_id]['name']})",
        task_type=TaskTypes.training,
    )
    # Log "model_variant" parameter to task
    task.set_parameter("model_variant", model_variant)
    task.connect_configuration(name="Dataset YAML", configuration=args['data'])
    task.connect_configuration(name="Dataset Content", configuration=dataset_content)

    # Load the YOLOv8 model
    model = YOLO(f'{model_variant}.pt')
    task.connect(args)

    # Train the model 
    # If running remotely, the arguments may be overridden by ClearML if they were changed in the UI
    try:
        results = model.train(**args)
    except Exception as e:
        raise e
    finally:
        task.close()
    return results

# Choose dataset

In [14]:
valid_choice = False
while not valid_choice:
    dataset_choice = '50e10f640d7548458d9c38ab9aac571b'#@param {type:"string"}
    if dataset_choice in env['DATASETS'].keys():
        valid_choice = True
    else:
        print("Dataset ID not found, please try again.")

# Model training

In [15]:
# Training inputs
model_variant = "yolov8n"
args = dict(
    epochs=100, 
    imgsz=480
)
results = training_task(model_variant, dataset_choice, args)


Original dataset:
{'names': {0: 'arc', 1: 'bar', 2: 'button', 3: 'buttonmatrix', 4: 'calendar', 5: 'chart', 6: 'checkbox', 7: 'dropdown', 8: 'image', 9: 'imagebutton', 10: 'keyboard', 11: 'label', 12: 'led', 13: 'line', 14: 'list', 15: 'menu', 16: 'messagebox', 17: 'roller', 18: 'scale', 19: 'slider', 20: 'spangroup', 21: 'spinbox', 22: 'spinner', 23: 'switch', 24: 'table', 25: 'tabview', 26: 'textarea', 27: 'tileview', 28: 'window'}, 'path': '/home/rini-debian/git-stash/lvgl-ui-detector/datasets', 'test': 'images/test', 'train': 'images/train', 'val': 'images/val'}
Adjusted dataset:
names:
  0: arc
  1: bar
  2: button
  3: buttonmatrix
  4: calendar
  5: chart
  6: checkbox
  7: dropdown
  8: image
  9: imagebutton
  10: keyboard
  11: label
  12: led
  13: line
  14: list
  15: menu
  16: messagebox
  17: roller
  18: scale
  19: slider
  20: spangroup
  21: spinbox
  22: spinner
  23: switch
  24: table
  25: tabview
  26: textarea
  27: tileview
  28: window
path: /home/rini-debia

train: Scanning /home/rini-debian/git-stash/lvgl-ui-detector/datasets/labels/train... 370 images, 0 backgrounds, 6 corrupt: 100%|██████████| 370/370 [00:00<00:00, 543.88it/s]

train: WARNING ⚠️ /home/rini-debian/git-stash/lvgl-ui-detector/datasets/images/train/ui_21.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0042]
train: WARNING ⚠️ /home/rini-debian/git-stash/lvgl-ui-detector/datasets/images/train/ui_236.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0063]
train: WARNING ⚠️ /home/rini-debian/git-stash/lvgl-ui-detector/datasets/images/train/ui_252.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0125]
train: WARNING ⚠️ /home/rini-debian/git-stash/lvgl-ui-detector/datasets/images/train/ui_294.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0187]
train: WARNING ⚠️ /home/rini-debian/git-stash/lvgl-ui-detector/datasets/images/train/ui_475.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0063]
train: WARNING ⚠️ /home/rini-debian/git-stash/lvgl-ui-detector/datasets/imag


val: Scanning /home/rini-debian/git-stash/lvgl-ui-detector/datasets/labels/val... 65 images, 0 backgrounds, 0 corrupt: 100%|██████████| 65/65 [00:00<00:00, 821.40it/s]

val: New cache created: /home/rini-debian/git-stash/lvgl-ui-detector/datasets/labels/val.cache


...

    100/100       1.5G     0.4612     0.6088     0.8656         34        480: 100%|██████████| 23/23 [00:04<00:00,  5.05it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  4.58it/s]

                   all         65        195      0.959      0.952      0.983      0.897



100 epochs completed in 0.162 hours.
Optimizer stripped from /home/rini-debian/git-stash/lvgl-ui-detector/runs/detect/train5/weights/last.pt, 6.2MB
Optimizer stripped from /home/rini-debian/git-stash/lvgl-ui-detector/runs/detect/train5/weights/best.pt, 6.2MB

Validating /home/rini-debian/git-stash/lvgl-ui-detector/runs/detect/train5/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.11.2 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 2060 with Max-Q Design, 6144MiB)
Model summary (fused): 168 layers, 3011303 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.28it/s]


                   all         65        195      0.959      0.951      0.983      0.903
                   arc         65         11      0.962          1      0.995      0.995
                   bar         65         19          1      0.809      0.978      0.829
                button         65          8      0.887      0.985      0.982      0.861
              calendar         65          7      0.943          1      0.995      0.995
              checkbox         65         15          1      0.778      0.941      0.798
              dropdown         65         15      0.954          1      0.995      0.966
                 label         65         20      0.939      0.765      0.922       0.62
                   led         65          8      0.963          1      0.995      0.878
                roller         65         22      0.979          1      0.995      0.919
                 scale         65         16      0.973          1      0.995      0.969
                slide

In [16]:
results.box.map

0.902833864538582

In [17]:
results.box.map50

0.9830693200725263

In [18]:
results.box.map75

0.9635186203486205

In [19]:
results.box.maps

array([      0.995,     0.82927,     0.86105,     0.90283,       0.995,     0.90283,     0.79785,     0.96603,     0.90283,     0.90283,     0.90283,     0.62012,     0.87767,     0.90283,     0.90283,     0.90283,     0.90283,     0.91869,     0.96924,     0.92874,     0.90283,     0.97235,     0.90283,     0.92165,
           0.98702,     0.90283,     0.90283,     0.90283,     0.90283])